# Продвинутое программирование на языке Python
## Семинар 1. Принципы ООП

На этом занятии постараемся разобрать логику `.__init__()` -> `.fit()` -> `.transform()` (`.predict()`). Это пригодится нам при работе с `sklearn`.

In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

In [2]:
X = np.random.uniform(-10, 10, (100, 2))
y = np.random.normal(0, 1, 100)

X_new = np.random.uniform(-10, 10, (50, 2))

In [3]:
lr = LinearRegression()  # .__init__()
# Y = b1*X1 + b2*X2 + ... + bn*Xn

In [4]:
type(lr)

sklearn.linear_model._base.LinearRegression

In [5]:
dir(lr)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_decision_function',
 '_estimator_type',
 '_get_default_requests',
 '_get_metadata_request',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_set_intercept',
 '_validate_data',
 '_validate_params',
 'copy_X',
 'fit',
 'fit_intercept',
 'get_metadata_routing',
 'get_params',
 'n_jobs',
 'positive',
 'predict',
 'score',
 'set_fit_request',
 's

In [6]:
lr.fit(X, y)

LinearRegression()

In [7]:
dir(lr)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_decision_function',
 '_estimator_type',
 '_get_default_requests',
 '_get_metadata_request',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_set_intercept',
 '_validate_data',
 '_validate_params',
 'coef_',
 'copy_X',
 'fit',
 'fit_intercept',
 'get_metadata_routing',
 'get_params',
 'intercept_',
 'n_features_in_',
 'n_jobs',
 'positive',
 

In [8]:
predict = lr.predict(X_new)

В процессе анализа данных нам часто нужно будет трансформировать их каким-либо образом. Рассмотрим две главных трансформации: стадартизацию и нормализацию.

![st&norm](https://i.stack.imgur.com/XmyWR.png)

Наша задача - реализовать класс `Transformer`.

При этом:
- для инициализации класса необходим вид трансформации;
- на вход может подаваться последовательность из целых / нецелых чисел (возможные виды: `list`, `tuple`, `np.ndarray`, `pd.DataFrame`). Последовательности (если это не `np.ndarray` или `pd.DataFrame`) должны иметь степень вложенности 2 (список списков);
- параметры для трансформации должны сохраняться в качестве атрибутов объекта (для каждой переменной);
- должна поддерживаться логика `fit`-`transform`.

In [62]:
class Transformer:
    _valid_ttypes = {'st', 'norm'}
    _valid_dtypes = {list, tuple, np.ndarray, pd.DataFrame}
    _valid_scalar_dtypes = {'int', 'float'}  # int32, int64 etc.
    
    def __init__(self, ttype='st'):
        if ttype not in self._valid_ttypes:
            raise ValueError(f'transformation type {ttype} is not recognized, use valid types: {self._valid_ttypes}')

        self.ttype = ttype

    def _validate_data(self, data):
        data_type = type(data)

        # check input types
        if data_type not in self._valid_dtypes:
            raise ValueError(f'input data type {data_type} is not recognized, use valid dtypes: {self._valid_dtypes}')

        if data_type in {list, tuple}:
            data = np.array(data)
        elif data_type == pd.DataFrame:
            data = data.values
        
        # check dtype
        scalar_dtype = str(data.dtype)
        dtype_condition = any(dt in scalar_dtype for dt in self._valid_scalar_dtypes)
        
        if not dtype_condition:
            raise ValueError(f'scalar data type {scalar_dtype} is not recognized, use valid types: {self._valid_scalar_dtypes}')

        # check shape
        if data.ndim != 2:
            raise ValueError(f'invalid dimensions number {data.ndim}, must be 2')

        return data

    def _transform(self, data):
        if self.ttype == 'st':
            data_transformed = (data - self.params['mean']) / self.params['std']
        else:
            data_transformed = (data - self.params['min']) / (self.params['max'] - self.params['min'])

        return data_transformed

    def fit_transform(self, data) -> np.ndarray:
        data_valid = self._validate_data(data)

        # get params
        if self.ttype == 'st':
            means = data_valid.mean(axis=0)
            stds = data_valid.std(axis=0)

            self.params = {
                'mean': means,
                'std': stds
            }
            
        else:
            minimums = data_valid.min(axis=0)
            maximums = data_valid.max(axis=0)

            self.params = {
                'min': minimums,
                'max': maximums
            }

        # apply params to old data and return
        return self._transform(data_valid) 

    def transform(self, new_data):
        if not hasattr(self, 'params'):
            raise ValueError('you cannot transform before you fit!')

        # validate
        new_data_valid = self._validate_data(new_data)

        # apply params to new data and return
        return self._transform(new_data_valid) 

In [67]:
tr = Transformer('st')

X_scaled = tr.fit(X)
X_new_scaled = tr.transform(X_new)

Теперь давайте попробуем немного изменить логику и реализовать родительский класс Transformer и два наследуемых: StandardScaler и NormScaler.

In [ ]:
class Transformer:
    _valid_dtypes = {list, tuple, np.ndarray, pd.DataFrame}
    _valid_scalar_dtypes = {'int', 'float'}  # int32, int64 etc.
    
    def __init__(self):
        pass

    def _validate_data(self, data):
        data_type = type(data)

        # check input types
        if data_type not in self._valid_dtypes:
            raise ValueError(f'input data type {data_type} is not recognized, use valid dtypes: {self._valid_dtypes}')

        if data_type in {list, tuple}:
            data = np.array(data)
        elif data_type == pd.DataFrame:
            data = data.values
        
        # check dtype
        scalar_dtype = str(data.dtype)
        dtype_condition = any(dt in scalar_dtype for dt in self._valid_scalar_dtypes)
        
        if not dtype_condition:
            raise ValueError(f'scalar data type {scalar_dtype} is not recognized, use valid types: {self._valid_scalar_dtypes}')

        # check shape
        if data.ndim != 2:
            raise ValueError(f'invalid dimensions number {data.ndim}, must be 2')

        return data

    def _get_params(self, data):  # in future _get_params method must be here
        pass

    def _apply_params(self, data):  # in future _apply_params method must be here
        pass

    def fit(self, data) -> np.ndarray:
        data_valid = self._validate_data(data)

        # get params and save
        self._get_params(data_valid)

    def transform(self, data):
        if not hasattr(self, 'params'):
            raise ValueError('you cannot transform before you fit!')

        # validate
        data_valid = self._validate_data(data)

        # apply params to new data and return
        data_transformed = self._apply_params(data_valid)
        
        return data_transformed

In [ ]:
class StandardScaler(Transformer):
    def _get_params(self, data):  # in future _get_params method must be here
        means = data.mean(axis=0)
        stds = data.std(axis=0)

        self.params = {
            'mean': means,
            'std': stds
        }

    def _apply_params(self, data):  # in future _apply_params method must be here
        data_transformed = (data - self.params['mean']) / self.params['std']

        return data_transformed